In [4]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
import pandas as pd
import os
import time

In [36]:
# Create train.txt
df = pd.read_csv('../../data_clean.csv')
df = df.sample(df.shape[0])

processed_list = list(map(lambda x: x.strip(), df['commentaire'] ))
with open("train.txt", 'w') as f:
  for comment in processed_list:
    f.write(comment + '\n')

In [37]:
path_to_file = './train.txt'
text = open(path_to_file, 'r').read()
print(text[:250])

Très bon travail et très bon résultat lors de l'évaluation finale. Félicitations !
Bonne volonté à l'oral mais le travail personnel est-il suffisant? Essayez de rester concentré en cours. Résultats moyens.
Un bon trimestre.
Un trimestre très fragile.


In [38]:
# Unique characters
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')
vocab

93 unique characters


['\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '8',
 '9',
 ':',
 ';',
 '<',
 '>',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'X',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '«',
 '°',
 '»',
 'À',
 'É',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '‘',
 '’']

## Process the text

In [39]:
# Convert strings to numerical representation
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)

In [40]:
# Recover chars from ids
chars_from_ids = preprocessing.StringLookup(
  vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [41]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


## Create training examples

In [42]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))

In [43]:
# Create dataset
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [44]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length+1)

In [45]:
# Create batches of length 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [46]:
# Create input and target from sequence
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [47]:
# Create dataset
dataset = sequences.map(split_input_target)

for input_text, target_text in dataset.take(1):
  print("Input: ", text_from_ids(input_text).numpy().decode('utf-8'))
  print("Target: ", text_from_ids(target_text).numpy().decode('utf-8'))

Input:  Très bon travail et très bon résultat lors de l'évaluation finale. Félicitations !
Bonne volonté à l
Target:  rès bon travail et très bon résultat lors de l'évaluation finale. Félicitations !
Bonne volonté à l'


In [48]:
# Shuffle and batch data
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (dataset
            .shuffle(BUFFER_SIZE)
            .batch(BATCH_SIZE, drop_remainder=True)
            .prefetch(tf.data.experimental.AUTOTUNE))
dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build model

In [49]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [50]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [51]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


In [54]:
# Test the model
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 100, 94) # (batch_size, sequence_length, vocab_size)


## Train

In [55]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)


In [56]:
model.compile(optimizer='adam', loss=loss)

In [57]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [77]:
EPOCHS = 3
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/3
112/112 [==============================] - 157s 1s/step - loss: 0.4525
Epoch 2/3
112/112 [==============================] - 164s 1s/step - loss: 0.4308
Epoch 3/3
112/112 [==============================] - 169s 2s/step - loss: 0.4104


## Generate text

In [59]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states


In [78]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [89]:
start = time.time()
states = None
next_char = tf.constant([' '])
result = [next_char]

for n in range(100):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)


 absences de  <name>  sont très passiveantés le niveau des exercices.
Un bon investissement tant à l' 

________________________________________________________________________________

Run time: 0.13953399658203125


## Save

In [76]:
tf.saved_model.save(one_step_model, 'one_step')


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [ ]:
# Load
one_step_reloaded = tf.saved_model.load('one_step')

In [69]:
# Use
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(150):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:  <name>  que vous auprontée du travail personnel, à l'écrit est très detter la partition en classe certinent. EllOT et Ona tentétions à conreux. C'es
